In [1]:
import openai
import os
from openai import OpenAI
import time
import json
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
openai.api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai.api_key)
model = "gpt-3.5-turbo-1106"

# Assistants API with Code Interpreter
Following docs here: https://platform.openai.com/docs/assistants/tools

In [3]:
# Create an assistant with Code Interpreter
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview"
)

In [4]:
# Create a thread
thread = client.beta.threads.create()

In [5]:
# Create a message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

In [6]:
# List messages
thread_messages = client.beta.threads.messages.list(thread.id)
print("\nRaw thread_messages.data\n", thread_messages.data)

for idx, message in enumerate(reversed(thread_messages.data)):
    print(f"\nMessage {idx}:\n", message.content[0].text.value)


Raw thread_messages.data
 [ThreadMessage(id='msg_rFhLSk6nEVNIJYrVIKIF46Wl', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1701465354, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_z6vg9PJTEIg5FFOQ3XTjW1mW')]

Message 0:
 I need to solve the equation `3x + 11 = 14`. Can you help me?


In [7]:
# Run the assistant
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account." # These instructions override the default instructions of the Assistant
)

# Check the run status. Expected output: in progress OR completed
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print("Run status: ", run.status)

Run status:  in_progress


In [9]:
# Check the run status again. Expected output: completed, move on to next step once completed
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print("Run status: ", run.status)

Run status:  completed


In [10]:
# Check the messages
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
print("\nRaw messages:\n", messages)

for idx, message in enumerate(reversed(messages.data)):
    print(f"\nMessage {idx}:\n", message.content[0].text.value)


Raw messages:
 SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_pMSFnyBIS2GcuwWLKgRmYBDh', assistant_id='asst_sm78sQgQCi0LF55h0ufEigwB', content=[MessageContentText(text=Text(annotations=[], value='The solution to the equation \\(3x + 11 = 14\\) is \\(x = 1\\).'), type='text')], created_at=1701465369, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_TQhXnWzEs5IU1T8oQIHwI0Pj', thread_id='thread_z6vg9PJTEIg5FFOQ3XTjW1mW'), ThreadMessage(id='msg_rFhLSk6nEVNIJYrVIKIF46Wl', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1701465354, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_z6vg9PJTEIg5FFOQ3XTjW1mW')], object='list', first_id='msg_pMSFnyBIS2GcuwWLKgRmYBDh', last_id='msg_rFhLSk6nEVNIJYrVIKIF46Wl', has_more=False)

Message 0:
 I need to solve the equation `3x + 11 = 14`. Can you

# Assistants API with Retrieval

In [11]:
file = client.files.create(
  file=open("test.txt", "rb"),
  purpose='assistants'
)

In [12]:
# Create an assistant with Retrieval
assistant = client.beta.assistants.create(
  name="Reader",
  description="You are great at reading files. You read the .txt files present. You also share a brief text summary of what you observed.",
  model="gpt-4-1106-preview",
  tools=[{"type": "code_interpreter", "type": "retrieval"}],
  file_ids=[file.id]
)

In [13]:
# Create a thread
thread = client.beta.threads.create()

In [14]:
# Create a message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Give me some summaries of the existing files"
)

In [15]:
# List messages
thread_messages = client.beta.threads.messages.list(thread.id)
print("\nRaw thread_messages.data\n", thread_messages.data)

for idx, message in enumerate(reversed(thread_messages.data)):
    print(f"\nMessage {idx}:\n", message.content[0].text.value)


Raw thread_messages.data
 [ThreadMessage(id='msg_6ZvaU32X0DJ7soRNWrkGPuNE', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Give me some summaries of the existing files'), type='text')], created_at=1701465399, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_pSTrW2TbwnmrOZT3bpuucO7M')]

Message 0:
 Give me some summaries of the existing files


In [16]:
# Run the assistant
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Check the run status. Expected output: in progress OR completed
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print("Run status: ", run.status)

Run status:  in_progress


In [18]:
# Check the run status again. Expected output: completed, move on to next step once completed
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print("Run status: ", run.status)

# Check the messages
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
print("\nRaw messages:\n", messages)

for idx, message in enumerate(reversed(messages.data)):
    print(f"\nMessage {idx}:\n", message.content[0].text.value)

assert(run.status == "completed")

Run status:  completed

Raw messages:
 SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_zsVC7e0mRTCPsGQSxIIbMRh9', assistant_id='asst_V4TVeoXG3OLIVGqsRZAwaGT3', content=[MessageContentText(text=Text(annotations=[], value='The existing file named "test.txt" contains a short message stating that it\'s a test file used for Assistants API. Additionally, the author of the file shares a fun fact about themselves, which is that they like grapes. Lastly, they mention that their special number is 100. The visible content of the file is 100% of its total content.'), type='text')], created_at=1701465405, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_G6u29fWcyWZEjEyY9oGWg5fs', thread_id='thread_pSTrW2TbwnmrOZT3bpuucO7M'), ThreadMessage(id='msg_6ZvaU32X0DJ7soRNWrkGPuNE', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Give me some summaries of the existing files'), type='text')], created_at=1701465399, file_ids=[], metadata

In [19]:
# Playing around with using an assistant file
file_2 = client.files.create(
  file=open("test_2.txt", "rb"),
  purpose='assistants'
)

In [20]:
# Create the assistant file
assistant_file = client.beta.assistants.files.create(
  assistant_id=assistant.id, 
  file_id=file_2.id
)
print(assistant_file)

AssistantFile(id='file-5LFx9bJZS6AF4OhjamP7dJsv', assistant_id='asst_V4TVeoXG3OLIVGqsRZAwaGT3', created_at=1701465443, object='assistant.file')


In [21]:
# Retrieve the assistant file
assistant_file = client.beta.assistants.files.retrieve(
  assistant_id=assistant.id, 
  file_id=file_2.id
)
print(assistant_file)

AssistantFile(id='file-5LFx9bJZS6AF4OhjamP7dJsv', assistant_id='asst_V4TVeoXG3OLIVGqsRZAwaGT3', created_at=1701465443, object='assistant.file')


In [22]:
# List assistant files
assistant_files = client.beta.assistants.files.list(
  assistant_id=assistant.id
)
print("# of files: ", len(assistant_files.data))
print(assistant_files)

# of files:  2
SyncCursorPage[AssistantFile](data=[AssistantFile(id='file-5LFx9bJZS6AF4OhjamP7dJsv', assistant_id='asst_V4TVeoXG3OLIVGqsRZAwaGT3', created_at=1701465443, object='assistant.file'), AssistantFile(id='file-jT0CpnGj6YHQlI4KwoCuMePa', assistant_id='asst_V4TVeoXG3OLIVGqsRZAwaGT3', created_at=1701465397, object='assistant.file')], object='list', first_id='file-5LFx9bJZS6AF4OhjamP7dJsv', last_id='file-jT0CpnGj6YHQlI4KwoCuMePa', has_more=False)


In [23]:
# Upload a file with an "assistants" purpose
file_3 = client.files.create(
  file=open("test_3.txt", "rb"),
  purpose='assistants'
)

In [24]:
# Run the Assistants API trying to retrieve the higher number

# Create a message
message = client.beta.threads.messages.create(
    thread_id=thread.id, # Continue the existing thread
    role="user",
    content="Tell me what the highest special number is based on the information in my files. I just added a new file.",
    file_ids=[file_3.id]
)

# List messages
thread_messages = client.beta.threads.messages.list(thread.id)
print("\nRaw thread_messages.data\n", thread_messages.data)

for idx, message in enumerate(reversed(thread_messages.data)):
    print(f"\nMessage {idx}:\n", message.content[0].text.value)


Raw thread_messages.data
 [ThreadMessage(id='msg_rR2KXE48PIlmRxkMOaInsvY2', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Tell me what the highest special number is based on the information in my files. I just added a new file.'), type='text')], created_at=1701465449, file_ids=['file-eV20bzfKgUk0aWkmQQOK0WyM'], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_pSTrW2TbwnmrOZT3bpuucO7M'), ThreadMessage(id='msg_zsVC7e0mRTCPsGQSxIIbMRh9', assistant_id='asst_V4TVeoXG3OLIVGqsRZAwaGT3', content=[MessageContentText(text=Text(annotations=[], value='The existing file named "test.txt" contains a short message stating that it\'s a test file used for Assistants API. Additionally, the author of the file shares a fun fact about themselves, which is that they like grapes. Lastly, they mention that their special number is 100. The visible content of the file is 100% of its total content.'), type='text')], created_at=1701465405, file_id

In [25]:
# Run the assistant
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Check the run status. Expected output: in progress OR completed
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print("Run status: ", run.status)

Run status:  in_progress


In [26]:
# Check the run status again. Expected output: completed, move on to next step once completed.
# Also added annotations here. # TODO: Current example doesn't use annotations, so improve the sample by increasing the size of the text files to support annotations
while run.status == "in_progress" or run.status == "queued":
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )
  print("Run status: ", run.status)
  time.sleep(1)

# Check the messages
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
print("\nRaw messages:\n", messages)

for idx, message in enumerate(reversed(messages.data)):
    # Adding annotations here

    # Extract the message content
    message_content = message.content[0].text
    annotations = message_content.annotations
    citations = []

    # Iterate over the annotations and add footnotes
    for index, annotation in enumerate(annotations):
        # Replace the text with a footnote
        message_content.value = message_content.value.replace(annotation.text, f' [{index}]')

        # Gather citations based on annotation attributes
        if (file_citation := getattr(annotation, 'file_citation', None)):
            cited_file = client.files.retrieve(file_citation.file_id)
            citations.append(f'[{index}] {file_citation.quote} from {cited_file.filename}')
        elif (file_path := getattr(annotation, 'file_path', None)):
            cited_file = client.files.retrieve(file_path.file_id)
            citations.append(f'[{index}] Click <here> to download {cited_file.filename}')
            # Note: File download functionality not implemented above for brevity

    # Add footnotes to the end of the message before displaying to user
    message_content.value += '\n' + '\n'.join(citations)
    
    print(f"\nMessage {idx}:\n", message_content)

assert(run.status == "completed")

Run status:  in_progress
Run status:  in_progress
Run status:  in_progress
Run status:  in_progress
Run status:  in_progress
Run status:  in_progress
Run status:  in_progress
Run status:  in_progress
Run status:  in_progress
Run status:  in_progress
Run status:  in_progress
Run status:  completed

Raw messages:
 SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_QV8UmZThgwemjrikh1jG4EE8', assistant_id='asst_V4TVeoXG3OLIVGqsRZAwaGT3', content=[MessageContentText(text=Text(annotations=[], value='The highest special number based on the information in your files is 250, mentioned in "test_3.txt" as the author\'s special number. For reference, "test.txt" mentioned a special number of 100, and "test_2.txt" needs to be checked for its special number to confirm, but since you have mentioned that you just added a new file ("test_3.txt"), we can assume that 250 is the highest so far, unless "test_2.txt" contains a higher number that we have not seen yet. Would you like me to check "test_2

In [27]:
# Delete an assistant file
deleted_assistant_file = client.beta.assistants.files.delete(
    assistant_id=assistant.id,
    file_id=file_2.id
)
print(deleted_assistant_file)

FileDeleteResponse(id='file-5LFx9bJZS6AF4OhjamP7dJsv', deleted=True, object='assistant.file.deleted')


In [28]:
# List assistant files
assistant_files = client.beta.assistants.files.list(
  assistant_id=assistant.id
)
print("# of files: ", len(assistant_files.data))
print(assistant_files)

# of files:  1
SyncCursorPage[AssistantFile](data=[AssistantFile(id='file-jT0CpnGj6YHQlI4KwoCuMePa', assistant_id='asst_V4TVeoXG3OLIVGqsRZAwaGT3', created_at=1701465397, object='assistant.file')], object='list', first_id='file-jT0CpnGj6YHQlI4KwoCuMePa', last_id='file-jT0CpnGj6YHQlI4KwoCuMePa', has_more=False)


In [29]:
# Cancelling a run. TODO: Integrate into sample somewhere to demonstrate.
# run = client.beta.threads.runs.cancel(thread_id=thread.id, run_id=run.id)
# while run.status == "in_progress":
#     run = client.beta.threads.runs.retrieve(
#         thread_id=thread.id,
#         run_id=run.id
#     )

# Assistants API with function calling

In [30]:
assistant = client.beta.assistants.create(
  instructions="You are a weather bot. Use the provided functions to answer questions.",
  model="gpt-4-1106-preview",
  tools=[{
      "type": "function",
    "function": {
      "name": "getCurrentWeather()",
      "description": "Get the weather in location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
          "unit": {"type": "string", "enum": ["c", "f"]}
        },
        "required": ["location"]
      }
    }
  }, {
    "type": "function",
    "function": {
      "name": "getNickname",
      "description": "Get the nickname of a city",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
        },
        "required": ["location"]
      }
    } 
  }]
)

In [31]:
# Create a thread
thread = client.beta.threads.create()

# Create a message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What's the weather for San Francisco?",
)

# Create a run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

In [37]:
# Check the run status. Expected output: in progress OR completed
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print("Run status: ", run.status)

Run status:  requires_action


In [33]:
# Functions hard coded for brevity
def getCurrentWeather(location, **kwargs):
    return "It's sunny!"

def getNickname(location):
    return "The city by the bay"

tools = {
    'getCurrentWeather': getCurrentWeather,
    'getNickname': getNickname
}

In [38]:
assert(run.status == "requires_action") # Needs to require action before calling tools

tool_outputs = []
for tool_call in run.required_action.submit_tool_outputs.tool_calls:
    tool_id, tool_function, tool_type = tool_call.id, tool_call.function, tool_call.type
    args = json.loads(tool_function.arguments)
    tool_output = tools[tool_function.name](**args)
    tool_outputs.append({
        "tool_call_id": tool_id,
        "output": tool_output
    })
print("tool_outputs\n", tool_outputs)

tool_outputs
 [{'tool_call_id': 'call_aeu20uVfjpUhyNSJsMqHFNwW', 'output': "It's sunny!"}]


In [39]:
run = client.beta.threads.runs.submit_tool_outputs(
  thread_id=thread.id,
  run_id=run.id,
  tool_outputs=tool_outputs
)

In [41]:
messages = client.beta.threads.messages.list(thread_id=thread.id)

print("\nRaw messages:\n", messages)

for idx, message in enumerate(reversed(messages.data)):
    print(f"\nMessage {idx}:\n", message.content[0].text.value)


Raw messages:
 SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_Qa7jhRGXte0lDoNf8A8kSRYz', assistant_id='asst_f5ccdm7Q3IugSjkacV5OdhX6', content=[MessageContentText(text=Text(annotations=[], value='The current weather in San Francisco, CA is sunny!'), type='text')], created_at=1701465546, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_JX7tkmn6SBAaxJ0Jb4dVoG5v', thread_id='thread_LbSNH8klIj5A2YDGgfAltIbx'), ThreadMessage(id='msg_u4rbcyKOw7hRykOrHx5pwOcb', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value="What's the weather for San Francisco?"), type='text')], created_at=1701465514, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_LbSNH8klIj5A2YDGgfAltIbx')], object='list', first_id='msg_Qa7jhRGXte0lDoNf8A8kSRYz', last_id='msg_u4rbcyKOw7hRykOrHx5pwOcb', has_more=False)

Message 0:
 What's the weather for San Francisco?

Message 1:
 The current weather in San Francisco, 

In [ ]:
# TODO: Add unit tests and ensure that they're comprehensive to ensure that the API shape between AOAI and OAI are the same. Likely use seed if supported.